<a href="https://colab.research.google.com/github/homenf/computer-vision/blob/master/assign1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from PIL import Image
import numpy as np
import math
from scipy import signal
import cv2

In [5]:
from IPython.display import Image

In [10]:
def boxfilter(n):
  if (n+1) % 2 :
    raise Exception("Sorry, no even number")
  return np.ones((n, n)) / n**2

boxfilter(3)
boxfilter(5)

array([[0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04]])

In [4]:
def gauss1d(sigma):
  length = np.floor(np.ceil(6 * sigma)/2)*2 + 1 # odd number
  dist = np.arange(length) - np.floor(length/2) # dist in pixels
  weight = np.exp(-(dist / sigma) **2 / 2)
  return weight/np.sum(weight)

gauss1d(2)

array([0.0022182 , 0.00877313, 0.02702316, 0.06482519, 0.12110939,
       0.17621312, 0.19967563, 0.17621312, 0.12110939, 0.06482519,
       0.02702316, 0.00877313, 0.0022182 ])

In [5]:
def gauss2d(sigma):
  oneD = gauss1d(sigma)
  filter = signal.convolve2d([oneD], np.transpose([oneD]))
  return filter / np.sum(filter)
gauss2d(1)

array([[1.96519161e-05, 2.39409349e-04, 1.07295826e-03, 1.76900911e-03,
        1.07295826e-03, 2.39409349e-04, 1.96519161e-05],
       [2.39409349e-04, 2.91660295e-03, 1.30713076e-02, 2.15509428e-02,
        1.30713076e-02, 2.91660295e-03, 2.39409349e-04],
       [1.07295826e-03, 1.30713076e-02, 5.85815363e-02, 9.65846250e-02,
        5.85815363e-02, 1.30713076e-02, 1.07295826e-03],
       [1.76900911e-03, 2.15509428e-02, 9.65846250e-02, 1.59241126e-01,
        9.65846250e-02, 2.15509428e-02, 1.76900911e-03],
       [1.07295826e-03, 1.30713076e-02, 5.85815363e-02, 9.65846250e-02,
        5.85815363e-02, 1.30713076e-02, 1.07295826e-03],
       [2.39409349e-04, 2.91660295e-03, 1.30713076e-02, 2.15509428e-02,
        1.30713076e-02, 2.91660295e-03, 2.39409349e-04],
       [1.96519161e-05, 2.39409349e-04, 1.07295826e-03, 1.76900911e-03,
        1.07295826e-03, 2.39409349e-04, 1.96519161e-05]])

In [16]:
# array ~ Dim[m*n*1]
# filter ~ Dim[w * h]
def convolve2d_manual(array, filter):
  # make inner most element an array for later broadcast
  newMat = [[0 for i in range(len(array[0]))] for j in range(len(array))]
  padLength = len(filter) // 2
  paddedArray = np.pad(array, ((padLength, padLength), (padLength, padLength)), 'constant', constant_values=0)
  for j in range(len(array)): # row
    for i in range(len(array[0])): # column
      neighbours = paddedArray[j : j+ 2*padLength+1, i: i+ 2* padLength+1]
      newMat[j][i] = np.sum(neighbours * filter, axis=(0,1))
  print(np.shape(newMat))
  return newMat


In [24]:
def gaussconvolve2d_manual(array, sigma):
  return convolve2d_manual(array, gauss2d(sigma))

def gaussconvolve2d_scipy(array,sigma):
  return signal.correlate2d(array, gauss2d(sigma), 'same')

def gaussconvolve2d_rgb(array, sigma):
  return convolve2d_rgb(array, gauss2d(sigma))

In [42]:
def convert_and_save(arr, name):
  arr = np.clip(np.array(arr), a_min = 0, a_max = 255)
  arr = arr.astype('uint8') 
  im = Image.fromarray(arr)
  im.save(name + '.png','PNG')  

def open_and_convert(name, toGrey=False):
  im = Image.open(name)
  im = im.crop((200,200,300,300))
  if toGrey:
    im = im.convert('L')
  im_array = np.asarray(im)
  return im_array.astype('float64')

im_array = open_and_convert('./dog.jpg', toGrey=True)
im_new_array = gaussconvolve2d_manual(im_array, 3)
convert_and_save(im_new_array, 'dog_manual')

im_array = open_and_convert('./dog.jpg', toGrey=True)
im_new_array = gaussconvolve2d_scipy(im_array, 3)
convert_and_save(im_new_array, 'dog_scipy')

(100, 100)


Part 3


In [29]:
# array ~ Dim[m*n*3]
# filter ~ Dim[w * h]
def convolve2d_rgb(array, filter):
  # make inner most element an array for later broadcast
  filter = [[[e] for e in row] for row in filter] 
  newMat = [[[0, 0, 0] for i in range(len(array[0]))] for j in range(len(array))]
  padLength = len(filter) // 2
  paddedArray = np.pad(array, ((padLength, padLength), (padLength, padLength), (0, 0)), 'constant', constant_values=0)
  for j in range(len(array)): # row
    for i in range(len(array[0])): # column
      neighbours = paddedArray[j : j+ 2*padLength+1, i: i+ 2* padLength+1]
      newMat[j][i] = np.sum(neighbours * filter, axis=(0,1))
  print(np.shape(newMat))
  return newMat

In [44]:
# Low frequency filtered image
low_array = open_and_convert('./dog.jpg', toGrey=False)
low_new_array = gaussconvolve2d_rgb(low_array, 3)
convert_and_save(low_new_array, 'dog_low_rgb')

# High frequency filtered image
high_array = open_and_convert('./0a_cat.bmp', toGrey=False)
high_new_array = gaussconvolve2d_rgb(high_array, 3) - high_array
convert_and_save(high_new_array + 128, 'cat_high_rgb') # add 128 for visualization only

# High + low
hybrid_array = low_new_array + high_new_array
convert_and_save(hybrid_array, 'hybrid_cat_dog')

(100, 100, 3)
(100, 100, 3)


Part 4

In [51]:
# ‘cv2.GaussianBlur’, ‘cv2.bilateralFilter’, and ‘cv2.medianBlur’. 
og_array = open_and_convert('./dog.jpg', toGrey=False)
cv2.GaussianBlur(og_array,(5,5),1)
convert_and_save(og_array, 'cv_rgb')
